In [24]:
!pip install langchain-community langchain-core
!pip install llama-cpp-python
!pip install langchain

In [31]:
from langchain import LlamaCpp, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory

In [32]:
llm = LlamaCpp(model_path="Phi-3-mini-4k-instruct-fp16.gguf", 
               n_gpu_layers=-1, 
               max_tokens=500, 
               n_ctx=2048, 
               seed=42,
               verbose=False
)

llama_new_context_with_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 32
llama_new_context_with_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_

In [33]:
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [34]:
basic_chain = prompt | llm

In [35]:
basic_chain.invoke({
    'input_prompt': 'Hi, My name is Hassan. What is 1 + 1 ?'
})

/Users/hassanraza/anaconda3/lib/python3.11/site-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


" 1 + 1 equals 2. It's a basic arithmetic addition problem!"

##### Multiple Prompts

In [36]:
template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.
<|end|>
<|assistant|>
"""
title_prompt = PromptTemplate(template=template, input_variables=['summary'])
title = LLMChain(llm=llm, prompt=title_prompt, output_key='title')

/var/folders/_b/drkg0vjx1v368n_9th3dhc7c0000gn/T/ipykernel_6173/4226334221.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  title = LLMChain(llm=llm, prompt=title_prompt, output_key='title')


In [37]:
title.invoke({"summary": "a girl that lost her mother"})

/Users/hassanraza/anaconda3/lib/python3.11/site-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Echoes of a Mother\'s Love: A Girl\'s Journey Through Loss"'}

In [38]:
template="""<s><|user|>
Describe the main character of story about {summary} with the title {title}.
User only two sentences. <|end|>
<|assistant|>
"""
character_prompt=PromptTemplate(template=template, input_variables=['summary', 'title'])
character=LLMChain(llm=llm, prompt=character_prompt, output_key='character')

In [39]:
template="""<s><|user|>
Create a stroy about {summary} with the title {title}. The nain character is: {character}. Only return the 
story and it cannot be longer than one paragraph. <|end|> 
<|assistant|>
"""
story_prompt=PromptTemplate(template=template, input_variables=['summary', 'title', 'character'])
story=LLMChain(llm=llm, prompt=story_prompt, output_key='story')

In [40]:
llm_chain = title | character | story
llm_chain.invoke("a girl that lost her mother")

/Users/hassanraza/anaconda3/lib/python3.11/site-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/Users/hassanraza/anaconda3/lib/python3.11/site-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/Users/hassanraza/anaconda3/lib/python3.11/site-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of a Mother\'s Love: A Tale of Healing"',
 'character': " The main character, Lily, is a compassionate and resilient young girl who grapples with the loss of her mother while learning to navigate life without her guiding presence. As she embarks on a journey towards healing and self-discovery, Lily draws strength from cherished memories of her mother's love and wisdom that continue to guide her through difficult times.",
 'story': ' "Whispers of a Mother\'s Love: A Tale of Healing" tells the story of Lily, a compassionate and resilient young girl who was forced to face the harsh reality of losing her mother. With each passing day without her guiding presence, Lily struggled with grief and confusion, but found solace in cherished memories that revealed the depths of her mother\'s love and wisdom. As she embarked on a journey towards healing and self-discovery, Lily learned to draw strength from these whispers of her mother

##### Conversation Buffer

In [ ]:
template="""<s><|user|> Current Conversation: {chat_history}
{input_prompt}<|end|>
<|assistant|>
"""
prompt=PromptTemplate(
    template=template,
    input_variables=['input_prompt', 'chat_history']
)

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [ ]:
llm_chain.invoke({"input_prompt": "Hi! My name is Hassan. What is 1 + 1?"})

In [ ]:
llm_chain.invoke({"input_prompt": "What is my name?"})

##### Summary Buffer

In [ ]:
summary_prompt_template = """<s><|user|>Summarize the
conversations and update with the new lines.

Current summary
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt=PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [ ]:
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

In [ ]:
llm_chain=LLMChain(
    prompt=prompt,
    llm=llm,
    memory
)

In [ ]:
llm_chain.invoke({
    "input_prompt": "Hi! My name is Hassan, What is 1+1?"
})
llm_chain.invoke({
    "input_propmt": "What is my name ?" 
})